In [1]:
import pandas as pd
import numpy as np
import zipfile

 <font color='blue'> To improve the presentation, let's import this class :

In [2]:
class color :
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

- In black : the teacher's instructions
- In blue : my remarks

----
###  Playing with all datasets

#### Loading dataset into dataframe objects

In [3]:
filenames=['data/ae_com.csv','data/amazon_com.csv','data/btemptd_com.csv','data/calvinklein_com.csv','data/hankypanky_com.csv','data/macys_com.csv','data/shop_nordstrom_com.csv','data/us_topshop_com.csv','data/victoriassecret_com.csv']

In [4]:
all_df=[pd.read_csv(f) for f in filenames]

Before having a look at the other dataset. 
To check for question 0 and 0bis, reload the df dataFrame from scratch 

**Question 1:** 

Check if for the Rp prices, we have a specific type of products sold. 

 <font color='blue'> 
 Let's generalize the function to any type of currency.
 The function will take as input value : 
- the dictionary matrix
- a function that can locate the currency
- the full name of the currency

In [5]:
def rp(x):
    return 'Rp'in x

In [6]:
def findcurrency(df_list,currency,name):
    for df in df_list:
        if (df['price'].apply(currency)).sum()>0:
            print (color.BOLD,df['retailer'][0]+' sells the following products in',name,':\n', color.END)
            for s in set(df.loc[df['price'].apply(currency)==True]['product_name']):
                print (s)

In [8]:
findcurrency(all_df,rp,"rupee")

 Victoriassecret US sells the following products in rupee :
 
PINK NEW! Lace Lightly Lined Halter Bralette
Easy Push-Up Bra
Dream Angels Demi Bra
Cotton Lingerie NEW! String Bikini Panty


<font color='blue'> 
2 types of products are sold in Rp : Bra and Bikini

**Question 2:**

Check for the factor to apply for currency converter to replace the prices by the USD equivalent. 

Consider that: 
- 1 IRD (Roupie) = 0.000071 USD 
- 1 Costa Rica dollar = 0.0016 USD

Create a function that 
- takes one argument to be one element from the price columns, 
- identifies which currency it is,
- extracts the value and converts it to float.

Then apply your function to both price columns in df. 

In [9]:
def convert_price(s):
    if s[:2]=='Rp':
        return float(s[2:])*0.000071
    elif s[0]=='₡':
        return float(s[1:])*0.0016
    elif s[-3:]=="USD":
        return float(s[:-3])
    elif '-'in s:
        from statistics import median
        return median([float(s[1:6]),float(s[-5:])])
    elif s[1]=='$':
        return float(s[2:6])
    else:
        return float(s[1:6])

 <font color='blue'>
    
Choice: 
- For the price ranges, we make the choice to replace with the median of the two ends
- For some lines, there is a discount price ("Now 10$"). Let's choose to take the original price and not the discount.

<font color='blue'> 
To go further, let's clean and homogenize the columns of each dictionary.
After looking at our dataset, we want ['price', 'mrp', 'rating', 'review_count'] in float and the rest in string.

   

In [10]:
def clean_dic(x):
    for df in x:    
        
        df[['price','mrp']]=df[['price','mrp']].applymap(convert_price)
        
        df[['rating','review_count']]=df[['rating','review_count']].astype('float64')
        
        medians = df[['price','mrp','rating','review_count']].median()
        
        df.fillna(value=medians,inplace=True)
        
        df['style_attributes']=df['style_attributes'].astype('object')

In [11]:
clean_dic(all_df)

In [12]:
[(dic['retailer'][0],dic.dtypes) for dic in all_df]

[('Ae US',
  product_name         object
  mrp                 float64
  price               float64
  pdp_url              object
  brand_name           object
  product_category     object
  retailer             object
  description          object
  rating              float64
  review_count        float64
  style_attributes     object
  total_sizes          object
  available_size       object
  color                object
  dtype: object),
 ('Amazon US',
  product_name         object
  mrp                 float64
  price               float64
  pdp_url              object
  brand_name           object
  product_category     object
  retailer             object
  description          object
  rating              float64
  review_count        float64
  style_attributes     object
  total_sizes          object
  available_size       object
  color                object
  dtype: object),
 ('Btemptd US',
  product_name         object
  mrp                 float64
  price               

 <font color='blue'> 
 By entering [dic.isna().sum() for dic in all_df] we see that there are still nan values for some dictionaries because ALL products are equal to nan, so it is the same for the median. We decide to leave these values in nan and not to replace them for the moment.

##### Question 3: 
Which online retailers sell some Victoria's Secret products? 

In [13]:
def f(x):
    return 'victoria' in x.lower()

 <font color='blue'>
  For the following questions, we will generalize the functions to apply them to all brands.
 Thus the functions will take as input values
    
 - our dictionary matrix
    
 - a predefined function to find the brand in a requested column
    
 - the name of the brand

In [14]:
def find_retailers(x,function,brand):
    for dic in x :
         if dic['brand_name'].apply(function).sum()>0:
            print(color.BOLD,dic['retailer'][0],color.END+"sells",dic['brand_name'].apply(function).sum(),brand,"products.\n")

In [15]:
find_retailers(all_df,f,"Victoria's Secret")

 Amazon US sells 67 Victoria's Secret products.

 Victoriassecret US sells 453386 Victoria's Secret products.



---
**Question 4:**
What are the Victoria's Secret products available in the different retailers?


In [16]:
def find_products(x,function,brand):
    for dic in x :
        if dic['brand_name'].apply(function).sum()>0:
            print(color.BOLD,dic['retailer'][0]+" sells these",brand,"products :\n",color.END)
            for s in set(dic.loc[dic['brand_name'].apply(function)==True]['product_name']):
                print(s+"\n")
            print()

In [17]:
find_products(all_df,f,"Victoria's Secret")

 Amazon US sells these Victoria's Secret products :
 
Victoria's Secret The Ultimate Maximum Support Sport Bra


 Victoriassecret US sells these Victoria's Secret products :
 
PINK Unlined Lace Multi-Way Bandeau

Cotton Lingerie Logo-waist Hiphugger Panty

Victoria Sport NEW! Lightweight by Victoria Sport Bra

The Lacie Lace Shortie Panty

PINK NEW! Shortie

Victoria Sport NEW! Knockout by Victoria Sport Front-Close Sport Bra

Very Sexy Fishnet Lace Bustier

Keyhole Push-Up Bra

PINK The Date Push-Up Bra

PINK NEW! Seamless Hipster

So Obsessed by Victoria's Secret Add-1½-Cups Push-Up Bra

Victoria Sport NEW! Lightweight by Victoria Sport Strappy Sport Bra

PINK Macrame One-Piece Halter

PINK NEW! Ultimate Push-Up Strappy Sports Bra

Victoria Sport Triangle Seamless Sport Bra

PINK NEW! Cropped Crew Tank

Very Sexy NEW! Lace & Satin Slip

PINK Mesh Halter Wrap Triangle Top

Body by Victoria Perfect Shape Bra

PINK NEW! Slouchy Full-Zip

Very Sexy NEW! Satin Midi Slip

Allover Lace from

--- 
**Question 5:**
How many distinct Victoria's Secret products do we have here? (check 'product_name')

<font color='blue'> 
1. Firstly, let's have a look to unique product by retailer

In [18]:
def count_product(x,function,brand):
    for dic in x :
        if dic['brand_name'].apply(function).sum()>0:
            print(dic['retailer'][0]+" sells",color.BOLD,len(set(dic.loc[dic['brand_name'].apply(function)==True]['product_name'])),color.END,"distinct",brand,"products.\n")

In [19]:
count_product(all_df,f,"Victoria's Secret")

Amazon US sells  1  distinct Victoria's Secret products.

Victoriassecret US sells  617  distinct Victoria's Secret products.



<font color='blue'> 
2. Secondly, let's have look at the unique products regardless the retailers

In [33]:
def total_product(x,function,brand):
    
    a=set()
    count=0
    
    for dic in x :
        
        if dic['brand_name'].apply(function).sum()>0:
            a.update(set(dic.loc[dic['brand_name'].apply(function)==True]['product_name']))
    
    count+=len(set(dic.loc[dic['brand_name'].apply(function)==True]['product_name']))
    
    print("The total",brand,"product number sold regardless the retailer is :",len(a))
    
    
    if len(a)<count:
        print("Retailers can therefore sell the same products.")
    else:
        print("The retailers sell different products.")

In [34]:
total_product(all_df,f,"Victoria's Secret")

Le nombre total de produits Victoria's Secret distincts (tout retailer confondu) est : 618
Les retailers vendent donc des produits différents.


**Question 6:**
Try to merge amazon and VictoriaSecret datasets considering only Victorias-Secret products, using merge method from Pandas. 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html?highlight=merge#pandas.DataFrame.merge

Compare prices.

<font color='blue'> 
For this question too, let's create a function that can be applied for all brands:

In [24]:
def merge_brand(x,function):
    l=[]
    for dic in x:
        if (dic['brand_name'].apply(function)).sum()>0:
            l.append(pd.DataFrame(dic[dic['brand_name'].apply(function)==True]))
    if len(l)==2: 
        return pd.merge(l[0],l[1],how='outer') #This block is not essential because we could also concat directly. But out of respect for the instruction, we merge.
    else:
        return pd.concat(l)

In [25]:
df_VS=merge_brand(all_df,f)

<font color='blue'> 
To check if the merging has been done, you can enter the following command

In [26]:
set(df_VS['retailer'])

{'Amazon US', 'Victoriassecret US'}

<font color='blue'>
As seen previously, the product from Amazon is not sold at Victoria's Secret. So there is no comparison to be made

---

**Question 7:**
Explore which brands ('brand_name') are sold by each retailer 

In [35]:
def brands(x):
    for dic in x:
        print(color.BOLD,dic['retailer'][0]+" sells these brands :\n",color.END)
        for s in set(dic['brand_name']):
            print(s+"\n")

In [36]:
brands(all_df)

 Ae US sells these brands :
 
AERIE

AEO

 Amazon US sells these brands :
 
b-temptd

Fila

Calvin-Klein

Sexy-Hair

Nintendo

Wacoal

ref=w_bl_sl_l_ap_ap_web_2586685011?ie=UTF8&node=2586685011&field-lbr_brands_browse-bin=Calvin+Klein

Creative-Motion

Compression-Comfort

s

ref=w_bl_sl_l_b_ap_web_2603426011?ie=UTF8&node=2603426011&field-lbr_brands_browse-bin=Wacoal

Hanky-Panky

Victorias-Secret

ref=w_bl_sl_l_b_ap_web_2586451011?ie=UTF8&node=2586451011&field-lbr_brands_browse-bin=b.tempt%27d

Lucky-Brand

 Btemptd US sells these brands :
 
WACOAL

 Calvin Klein US sells these brands :
 
Calvin Klein

 Hankypanky US sells these brands :
 
HankyPanky

 Macys US sells these brands :
 
Calvin Klein

Wacoal

Hanky Panky

Calvin Klein Modern Cotton

Calvin Klein Performance

b.tempt'd

Vanity Fair

b.tempt'd by Wacoal

 Nordstrom US sells these brands :
 
CALVIN KLEIN

B.TEMPT'D BY WACOAL

WACOAL

NORDSTROM LINGERIE

HANKY PANKY

 Topshop US sells these brands :
 
US TOPSHOP

 Victoriasse

---

**Question 8:**
As we do not have so much Victoria's Secret products in the different files.

Explore the different files and the content for Calvin Klein products. 

Which retailers provide Calvin Klein products?


In [37]:
def h(x):
    return 'calvin' in x.lower()

In [38]:
find_retailers(all_df,h,"Calvin Klein")

 Amazon US sells 7373 Calvin Klein products.

 Calvin Klein US sells 4747 Calvin Klein products.

 Macys US sells 15989 Calvin Klein products.

 Nordstrom US sells 3142 Calvin Klein products.



----
**Question 9: Extraction and cleaning of Calvin Klein products data** 

Extract all data from each file for the Calvin Klein products. Check for the brand_name. It could be written in multiple ways. 

Put each in new dataframes. 

To do so: 
- Consider dataframes containing Clavin Klein products and preprocess each dataframe separately (it might be useful to adjust your preprocessing function when content is different)
- Identify and extract Calvin Klein product data

Use the concat method to compile all extracted data into a new df_ck data frame. Also clean the price columns. Have a close look to the price columns.

You can also apply the fillna method applying medians (to be calculated again).

In [39]:
df_CK=merge_brand(all_df,h)

In [40]:
df_CK.isna().sum()

product_name           0
mrp                    0
price                  0
pdp_url                0
brand_name             0
product_category       0
retailer               0
description            0
rating              4747
review_count        4747
style_attributes       0
total_sizes            0
available_size         0
color                  0
dtype: int64

<font color='blue'> 
Let's replace the missing values with the median from the df_ck dictionary

In [41]:
medians1=df_CK[['rating','review_count']].median()
df_CK.fillna(value=medians1,inplace=True)

In [42]:
find_retailers(all_df,h,'Calvin Klein')

 Amazon US sells 7373 Calvin Klein products.

 Calvin Klein US sells 4747 Calvin Klein products.

 Macys US sells 15989 Calvin Klein products.

 Nordstrom US sells 3142 Calvin Klein products.



In [43]:
find_products(all_df,h,'Calvin Klein')

 Amazon US sells these Calvin Klein products :
 
Calvin Klein Women's 4 Pack Stretch Lace Hipster Panty

Calvin Klein Women's Id Cotton Large Waistband Triangle Unlined Bra

Calvin Klein Women's ID Tanga Wide Waistband Cotton Panty

Calvin Klein Women's Pure Seamless Bikini Panty (Pack of 3)

Calvin Klein Women's Ck One Cotton Unlined Bralette

Calvin Klein Women's Horizon Seamless 2 Pack Bralette

Calvin Klein Women's Standard Ck One Cotton Bikini Panty

Calvin Klein Women's Carousel 3 Pack Thong

Calvin Klein Women's Everyday Lightly Lined Demi Bra

Calvin Klein Women's Modern Cotton Bikini

Calvin Klein Women's 3 Pack Carousel Thong Panty

Calvin Klein Women's Id Cotton Large Waistband Tanga Panty

Calvin Klein Women's Perfectly Fit Lightly Lined Memory Touch T-Shirt Bra

Calvin Klein Women's Micro Hipster Panty

Calvin Klein Women's Everyday Calvin Full Coverage Lightly Lined Bra

Calvin Klein Women's Modern Cotton Boyshort Panty

Calvin Klein Women's Seductive Comfort Tailored Cus

In [44]:
count_product(all_df,h,'Calvin Klein')

Amazon US sells  40  distinct Calvin Klein products.

Calvin Klein US sells  107  distinct Calvin Klein products.

Macys US sells  211  distinct Calvin Klein products.

Nordstrom US sells  87  distinct Calvin Klein products.



In [45]:
total_product(all_df,h,'Calvin Klein')

Le nombre total de produits Calvin Klein distincts (tout retailer confondu) est : 445
Les retailers vendent donc des produits différents.


<font color='blue'> 
There are no prices to compare this time either.

**Question 10:**

Consider your Calvin Klein dataFrame. 

Extract the description of every product, only keep the unique description, and count which words appear the most. 

<font color='blue'> 
Let's make the choice here not to differentiate between capitalized and non-capitalized words. Thus, we will consider "Calvin" or "CALVIN" as one and the same word. Moreover, we will remove the punctuation so as not to distort the count

In [46]:
from collections import Counter

In [47]:
def description(dic,top_words):
    a=[]
    for s in set(dic['description']):
        a.extend(s.lower().strip('.').split(' '))
    return Counter(a).most_common(top_words)

In [48]:
description(df_CK,10)

[('a', 469),
 ('and', 348),
 ('the', 263),
 ('calvin', 246),
 ('with', 206),
 ('klein', 206),
 ('this', 202),
 ('logo', 170),
 ('from', 161),
 ('in', 143)]

**Question 11:**

Consider all dataframes (you might concat them all into one after having processed them separately).

- which categories of products are sold most?
- which products are sold most?
- which products have the highest ratings?
- what are the top sellers (sold most and best ratings)?
- what is the top product color (sold most and best ratings)?

In [50]:
df_complete=pd.concat(all_df)

<font color='blue'>
Before starting questions 11 and 12 :
- replace the missing values by the global median of our dictionary
- let's homogenize the columns 'brand_name' and 'product_category'. Indeed, depending on the retailer there are several ways to refer to a brand or a category.

In [51]:
df_complete.isna().sum()

product_name             0
mrp                      0
price                    0
pdp_url                  0
brand_name               0
product_category         0
retailer                 0
description              0
rating                8265
review_count          8265
style_attributes    456468
total_sizes              0
available_size           0
color                  677
dtype: int64

In [52]:
medians2=df_complete[['rating','review_count']].median()
df_complete.fillna(value=medians2,inplace=True)

<font color='blue'> 
For the brands:

<font color='blue'> 
The choice will be made to group subsidiaries under the same brand. Thus "aeo" and "aerie" will be grouped under the brand American Eagle, "b.tempt" will join the brand Wacoal Corporation, etc.

 <font color='blue'>    
Moreover, after having advanced in the questions, we realize that the brand_name does not always correspond to the products in the amazon dictionary!

 <font color='blue'>    
For example: the products of brands Nintendo, s, Compression-Comfort, Creative-Motion, Fila, Lucky-Brand, Sexy-Hair are in fact Wacoal products.

 <font color='blue'>    
To check it, you just have to enter for each brand mentioned above: df_complete.loc[df_complete['brand_name'] == 'Nintendo']. Clicking on the url you can see the brand error.

In [53]:
def clean_brand(x):
    if 'ae' in x.lower():
        return 'American Eagle'
    elif 'calvin' in x.lower():
        return 'Cavlin Klein'
    elif'hanky' in x.lower():
        return 'Hanky Panky'
    elif 'victoria' in x.lower():
        return "Victoria's Secret"
    elif 'nordstrom' in x.lower():
        return 'Nordstrom Lingerie'
    elif 'topshop' in x.lower():
        return "US Topshop"
    elif "vanity" in x.lower():
        return "Vanity Fair"
    else:
        return 'Wacoal Corporation'

In [54]:
def clean_cat(x):

    if 'bikini' in x.lower() :
        return 'Bikinis'
    elif 'bridal' in x.lower() or'bridget' in x.lower() or 'brooke' in x.lower() or 'audrey' in x.lower() or 'collections' in x.lower() or 'cheekies' in x.lower() or 'chill' in x.lower() or 'classic' in x.lower() or 'hannah' in x.lower() or 'katie' in x.lower():
        return 'Special collections'
    elif 'crop' in x.lower() or 'kimono' in x.lower() or 'onepiece' in x.lower() or 'one piece' in x.lower() or 'one-piece' in x.lower() or 'cross-back scoop'in x.lower() or 'body'in x.lower() or 'bodies'in x.lower() or 'tee' in x.lower() or 'tank' in x.lower() or 'top' in x.lower() or 'trim' in x.lower() or 'wrap' in x.lower() or 'neck' in x.lower():
        return 'Top'
    elif 'bottle' in x.lower() or 'kit' in x.lower() or 'tote' in x.lower() or 'hat' in x.lower() or 'sock' in x.lower():
        return 'Accessories'
    elif 'sleep' in x.lower() or 'night' in x.lower():
        return 'Sleepwear'
    elif 'sport' in x.lower() or 'gym' in x.lower():
        return 'Sport'
    else:
        return 'Underwear'

In [55]:
df_complete['brand_name']=df_complete['brand_name'].apply(clean_brand)
df_complete['product_category']=df_complete['product_category'].apply(clean_cat)

<font color='blue'> 
1. Which categories of products are sold most?
   

<font color='blue'> 
Let's assume that the more a product is purchased, the more reviews it is likely to get. 

<font color='blue'> 
So let's consider that the best-selling categories are those whose products have the most reviews.

In [57]:
def best_category(df,top_number): #fonction qui prend en valeurs d'entrée un dictionnaire et le TOP qu'on veut avoir
    
    df1 =df.drop_duplicates(subset=['pdp_url'])[['product_category','review_count']]#Partons du principe que sur chaque url il y a des reviews différentes. On supprime les url en double pour ne pas compter 2 fois une review.
    m=[]
    
    for cat in set(df1['product_category']):
        m.append((cat, df1.loc[df1['product_category'].str.contains(cat,regex=False)]['review_count'].sum()))
    
    
    print(color.BOLD,"\nThe top",top_number,"most sold product categories is :\n",color.END)    
    
    count=1
    for name,review in sorted(m, key=lambda item: item[1],reverse=True)[:top_number]:
        print(count,"-",name)
        count+=1

In [58]:
best_category(df_complete,5)

 
The top 5 most sold product categories is :
 
1 - Underwear
2 - Sport
3 - Top
4 - Special collections
5 - Sleepwear


<font color='blue'> 
2. Which products are sold most?
    
Let's repeat the same reasoning but this time replacing the category by the product.
As there are many cases of equality, we also classify by alphabetical order

In [59]:
def best_product(df,top_number): 
    
    df2 =df.drop_duplicates(subset=['pdp_url'])[['product_name','review_count']] 
    s=[]
    
    for product in set(df2['product_name']):
        s.append((product, df2.loc[df2['product_name'].str.contains(product,regex=False)]['review_count'].sum()))
        
    print(color.BOLD,"\nThe top",top_number,"most sold products is :\n",color.END)    
    
    count=1
    for product,review in sorted(s, key=lambda item: (-item[1],item[0]))[:top_number]:
        print(count,"-",product)
        count+=1

In [60]:
best_product(df_complete,5)

 
The top 5 most sold products is :
 
1 - Thong
2 - Allover Lace from Cotton Lingerie NEW! Dotted Mesh Thong Panty
3 - Body by Victoria NEW! Floral Lace Mix Cheeky Panty
4 - Cotton Lingerie Lace-waist Thong Panty
5 - Cotton Lingerie NEW! Dotted Mesh Thong Panty


<font color='blue'>
3. Which products have the highest ratings?


<font color='blue'>
To do this, let's average the ratings for each product and classify the products according to 2 criteria
1. the rating
2. the alphabetical order

In [61]:
def best_rating(df,top_number):
    
    df3=df.drop_duplicates(subset=['pdp_url'])[['product_name','rating']]
    l=[]
    
    for product in set(df3['product_name']):
        l.append((product, df3.loc[df3['product_name'].str.contains(product,regex=False)]['rating'].mean()))
  

    print(color.BOLD,"\nThe top",top_number,"best rated products is :\n",color.END)  

    count=1
    for product,rate in sorted(l, key=lambda item: (-item[1],item[0]))[:top_number]:
        print(count,"-",product)
        print("rate:",rate)
        count+=1

In [62]:
best_rating(df_complete,10)

 
The top 10 best rated products is :
 
1 - "I DO" Signature Lace Cheeky Hipster with Gift Box
rate: 5.0
2 - 'After Midnight' Open Gusset Lace Thong
rate: 5.0
3 - 'B Smooth' High Cut Briefs
rate: 5.0
4 - 'B Smooth' Seamless Briefs
rate: 5.0
5 - 'CK One' Thong
rate: 5.0
6 - 'Ciao Bella' Bralette
rate: 5.0
7 - 'I Do' Swarovski Crystal Original Rise Thong
rate: 5.0
8 - 'Lace Affair' Tanga Briefs
rate: 5.0
9 - 'Lace Affair' Underwire Balconette Bra
rate: 5.0
10 - 'Logo' Low Rise Thong
rate: 5.0


<font color='blue'>
4. What are the top sellers (sold most and best ratings)?


<font color='blue'>
To do this, let's average the ratings for each product. 


Then let's rank our products according to 2 criteria: 
- the highest rating
- the number of reviews

<font color='blue'>
We will put higher in the ranking, for 2 products with the same rating, the one with the most reviews.

In [63]:
def best_seller(df,top_number):
    df4 =df.drop_duplicates(subset=['pdp_url'])[['product_name','rating','review_count']]
    x=[]
    
    for product in set(df4['product_name']):
        x.append((product, df4.loc[df4['product_name'].str.contains(product,regex=False)]['rating'].mean(), df4.loc[df4['product_name'].str.contains(product,regex=False)]['review_count'].sum()))


    print(color.BOLD,"\nThe top",top_number,"seller products are :\n",color.END)

    count=1
    for name,rate,review in sorted(x, key=lambda item: (item[1],item[2]),reverse=True)[:top_number]:
        print(count,"-",name)
        print("overall rating :",rate)
        print("total reviews :",review)
        count+=1

In [64]:
best_seller(df_complete,10)

 
The top 10 seller products are :
 
1 - Aerie Everyday Loves Lace Cheeky
overall rating : 5.0
total reviews : 116.0
2 - Embrace Lace Bikini 64391
overall rating : 5.0
total reviews : 93.0
3 - Signature Lace Cheeky Hipster
overall rating : 5.0
total reviews : 32.0
4 - BARE "Eve" Natural Rise Thong
overall rating : 5.0
total reviews : 20.0
5 - 3 Pack Low Rise Thongs
overall rating : 5.0
total reviews : 20.0
6 - 3 Pack Original Rise Thongs
overall rating : 5.0
total reviews : 20.0
7 - Calvin Klein Women's Ombre 5 Pack Thong
overall rating : 5.0
total reviews : 19.0
8 - Lingerie Wash Bag
overall rating : 5.0
total reviews : 18.0
9 - Aerie Shine Boyshort + Boho Lace
overall rating : 5.0
total reviews : 17.0
10 - Thong F3786
overall rating : 5.0
total reviews : 16.0


<font color='blue'>
5. What is the top product color (sold most and best ratings)?

<font color='blue'>
To do this, we will use exactly the same reasoning as for the products but this time for the colors


In [65]:
def best_color(df,top_number):
    df5 =df.drop_duplicates(subset=['pdp_url'])[['color','rating','review_count']].dropna(subset=['color'])
    c=[]
    
    for couleur in set(df5['color']):
        c.append((couleur, df5.loc[df5['color'].str.contains(couleur,regex=False)]['rating'].mean(), df5.loc[df5['color'].str.contains(couleur,regex=False)]['review_count'].sum()))


    print(color.BOLD,"\nThe top",top_number,"colors is :\n",color.END)

    count=1
    for col,rate,review in sorted(c, key=lambda item: (item[1],item[2]),reverse=True)[:top_number]:
        print(count,"-",col)
        print("overall rating :",rate)
        print("total reviews :",review)
        count+=1

In [66]:
best_color(df_complete,10)

 
The top 10 colors is :
 
1 - Leopard Nouveau
overall rating : 5.0
total reviews : 98.0
2 - Violet Spray
overall rating : 5.0
total reviews : 63.0
3 - White/Powder Blue
overall rating : 5.0
total reviews : 52.0
4 - Sophisticat
overall rating : 5.0
total reviews : 48.0
5 - Capri Bloom
overall rating : 5.0
total reviews : 48.0
6 - Black/ Blue
overall rating : 5.0
total reviews : 42.0
7 - Bellisima
overall rating : 5.0
total reviews : 42.0
8 - Aloha
overall rating : 5.0
total reviews : 40.0
9 - Moody Blooms
overall rating : 5.0
total reviews : 40.0
10 - Check Please
overall rating : 5.0
total reviews : 38.0


**Question 12:**

Consider all dataframes (you might concat them all into one after having processed them separately).

- how can you assess an overall rating for a brand based on the ratings of its products?
- according to your definition, which brands have the best ratings?
- which types of products sold by all brands? (you might have to process the product category strings) and which product of these categories are sold by all brands?
- among these products, which brands has the best ratings?

<font color='blue'>
    1.How can you assess an overall rating for a brand based on the ratings of its products?

To have access to a rating of a brand, you have to make the average of the notes received by all the products of the brand by taking care to remove the duplicate url to not count twice a note.
  <font color='blue'>  
    2.According to your definition, which brands have the best ratings?

In [67]:
def best_brand(df,top_number):

    df5=df.drop_duplicates(subset=['pdp_url'])[['brand_name','rating']]
    means=[]
    
    for brand in set(df5['brand_name']):
        means.append((brand, df5.loc[df5['brand_name'].str.contains(brand,regex=False)]['rating'].mean()))
    
    print(color.BOLD,"\nThe top",top_number,"brands are :\n",color.END)
    
    count=1
    for name,rate in sorted(means, key=lambda item: item[1],reverse=True)[:top_number]:
        print(count,"-",name)
        print("overall rating :",round(rate, 2))
        count+=1

In [68]:
best_brand(df_complete,8)

 
The top 8 brands are :
 
1 - Hanky Panky
overall rating : 4.83
2 - US Topshop
overall rating : 4.66
3 - Vanity Fair
overall rating : 4.49
4 - Cavlin Klein
overall rating : 4.32
5 - Wacoal Corporation
overall rating : 4.31
6 - American Eagle
overall rating : 4.25
7 - Victoria's Secret
overall rating : 4.24
8 - Nordstrom Lingerie
overall rating : 3.68


<font color='blue'>3. Which types of products sold by all brands? (you might have to process the product category strings)

In [69]:
def common_cat(df):
    df6=df[['brand_name','product_category']].drop_duplicates()

    for product in set(df6['product_category']):
        if list(df6['product_category']).count(product) == len(set(df6['brand_name'])):
            print(product+" category is sold by all brands\n") 

# reasoning :
#First we extract a dictionary with the unique "duo" (brand name- product category)
#If the number of times a category appears in our new dictionary is equal to the number of brands 
#then it is present for each brand

In [70]:
common_cat(df_complete)

Underwear category is sold by all brands



In [71]:
def common_product(df,cat):
    
    df7=df.loc[df['product_category'].str.contains(cat,regex=False)][['brand_name','product_name']].drop_duplicates()
    l=[]
    for product in set(df7['product_name']):
        if list(df7['product_name']).count(product) == len(set(df7['brand_name'])):
            l.append(product)
            print(product+"is sold by all brands\n")
    if len(l)==0:
        print("No product in "+cat+" is sold by all brands")

In [72]:
common_product(df_complete,'Underwear')

No product in Underwear is sold by all brands


<font color='blue'>
In general, no product is sold by everyone

<font color='blue'>
4. Among these products, which brands has the best ratings?

For each product category, let's calculate which brands have the highest scores.

In [73]:
def best_percat(df,top_number):
    
    df8=df.drop_duplicates(subset=['pdp_url'])[['brand_name','product_category','rating']]
   
    for cat in set(df8['product_category']):
        l=[]
        df8bis=df8.loc[df8['product_category'].str.contains(cat,regex=False)]
        
        print(color.BOLD,"\n For the",cat,"category, the best brands are :\n",color.END)               
        
        
        for brand in set(df8bis['brand_name']):
            
            l.append((brand,df8bis.loc[df8bis['brand_name'].str.contains(brand,regex=False)]['rating'].mean()))
            
        
        count=1
        for brand,rate in sorted(l, key=lambda item: item[1],reverse=True)[:top_number]:
            print(count,'-',brand)
            print('rating :',round(rate,2))
            count+=1

In [75]:
best_percat(df_complete,8)

 
 For the Underwear category, the best brands are :
 
1 - Hanky Panky
rating : 4.72
2 - US Topshop
rating : 4.66
3 - Vanity Fair
rating : 4.49
4 - Cavlin Klein
rating : 4.32
5 - Wacoal Corporation
rating : 4.31
6 - Victoria's Secret
rating : 4.24
7 - American Eagle
rating : 4.24
8 - Nordstrom Lingerie
rating : 3.63
 
 For the Special collections category, the best brands are :
 
1 - Hanky Panky
rating : 4.99
2 - Cavlin Klein
rating : 4.4
3 - Wacoal Corporation
rating : 4.35
4 - American Eagle
rating : 4.28
5 - Nordstrom Lingerie
rating : 3.81
6 - Victoria's Secret
rating : 3.4
 
 For the Top category, the best brands are :
 
1 - Hanky Panky
rating : 4.98
2 - American Eagle
rating : 4.47
3 - Victoria's Secret
rating : 4.25
 
 For the Bikinis category, the best brands are :
 
1 - American Eagle
rating : 4.33
2 - Victoria's Secret
rating : 4.31
 
 For the Sport category, the best brands are :
 
1 - American Eagle
rating : 4.17
2 - Victoria's Secret
rating : 4.17
 
 For the Accessories ca